###  ABS to Tidydata

In [1]:
from databaker.framework import *
import pandas as pd 

ABS Excel spreadsheet is available from [ONS website](https://www.ons.gov.uk/businessindustryandtrade/business/businessservices/datasets/annualbusinesssurveyimportersandexporters).

In [2]:
import requests
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/business/businessservices/datasets/annualbusinesssurveyimportersandexporters/current/importersandexporterssummarytablesinitial.xls'
inputFile = sourceFolder / 'importersandexporterssummarytablesinitial.xls'
if not(inputFile.exists() and inputFile.is_file()):
    response = requests.get(inputURL)
    with open(inputFile, 'wb') as f:
        f.write(response.content)

In [3]:
Filenames = [
    'goods_and_services_provisional_2016',
    'goods_provisional_2016',
    'services_provisional_2016',
    'goods_and_services_revised_2015',
    'goods_revised_2015',
    'services_revised_2015',
    'goods_and_services_revised_final_2014',
    'goods_revised_final_2014',
    'services_revised_final_2014'
]

In [4]:
tab_main = loadxlstabs(inputFile)

Loading in/importersandexporterssummarytablesinitial.xls which has size 109867 bytes
Table names: ['Content Page', '2016 Goods and Services', '2016 Goods ', '2016 Services  ', '2015 Goods and Services', '2015 Goods ', '2015 Services  ', '2014 Goods and Services', '2014 Goods', '2014 Services', 'Standard Errors']


In [5]:
destinationFolder = Path('out') / 'prep'
destinationFolder.mkdir(exist_ok=True, parents=True)

for sh in list(range(0,9)):
    
    tab = tab_main[sh+1]
    observations = tab.excel_ref('B14').expand(RIGHT).expand(DOWN).is_not_blank()- tab.excel_ref('B70').expand(DOWN)
    Trade = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
    CET = tab.excel_ref('A').expand(DOWN).by_index([12,26,36,45,61]).is_not_blank()
    segments = tab.excel_ref('A').expand(DOWN).is_not_blank()- CET - tab.excel_ref('A70').expand(DOWN)-tab.excel_ref('A10').expand(ABOVE)
    measure = tab.excel_ref('B11:F11').expand(RIGHT).is_not_blank()
    Dimensions = [
                HDimConst('Geography', 'K02000001'),
                HDimConst('TIME', '2016'),
                HDim(CET,'Count, Employment and Turnover',CLOSEST,ABOVE),
                HDim(measure,'Measure', CLOSEST, LEFT), 
                HDim(segments,'Statistics', DIRECTLY, LEFT ),
                HDim(Trade,'Flow',DIRECTLY,ABOVE)

    ] 
    c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
    new_table = c1.topandas()
    new_table['Flow'] = new_table['Flow'].str.rstrip('47')
    new_table['Count, Employment and Turnover'] = new_table['Count, Employment and Turnover'].str.rstrip('34')
    new_table['Measure'] = new_table['Measure'].str.rstrip('56')
    new_table['Measure'] = new_table['Measure'].map(lambda cell:cell.replace('of', ''))
    new_table['Measure'] = new_table['Measure'].map(lambda cell:cell.replace('%', 'Percentage'))
    new_table['Statistics'] = new_table['Statistics'].astype(str)
    new_table['Statistics'] = new_table['Statistics'].map(lambda cell:cell.replace('+', ' Above'))
    new_table['Count, Employment and Turnover'] = new_table['Count, Employment and Turnover'].str.replace('3','')
    new_table['Period'] = new_table['TIMEUNIT'].map(str) + '/' + new_table['TIME']
    new_table = new_table.drop('TIMEUNIT', axis=1)
    new_table = new_table.drop('DATAMARKER', axis=1)
    new_table = new_table.drop('TIME', axis=1)
    new_table = new_table[new_table['OBS'] != '']
    new_table =new_table[['Geography','Period','Flow','Count, Employment and Turnover','Statistics','OBS','Measure']]
    new_table.to_csv(destinationFolder / (Filenames[sh]+'.csv'), index = False)


TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
